<a href="https://colab.research.google.com/github/FahmidaNourin/Light_Sleep_And_Deep_Sleep_Classification/blob/main/Project_Initial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#connecting with drive

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#read in csv file

In [32]:
import numpy as np
import pandas as pd


df = pd.read_csv("/content/drive/MyDrive/Sleep_Research/HMC_Sleep_Stages_75_Subject.csv")

In [33]:
df.head()

,Subject,Sleep_Stage,Epoch,MeanP_Alpha_F4,MedianF_Alpha_F4,MeanF_Alpha_F4,SpectralEdge_Alpha_F4,PeakF_Alpha_F4,MeanP_Beta_F4,MedianF_Beta_F4,...,MeanP_Delta_O2,MedianF_Delta_O2,MeanF_Delta_O2,SpectralEdge_Delta_O2,PeakF_Delta_O2,MeanP_Gamma_O2,MedianF_Gamma_O2,MeanF_Gamma_O2,SpectralEdge_Gamma_O2,PeakF_Gamma_O2
0,SN002,N1,6,0.00034,10.11512,17.85756,17.48293,10.24000,0.00047,17.73268,...,0.00112,0.74927,6.36878,2.87220,0.62439,0.000076,32.71805,43.83220,49.95122,49.95122
1,SN002,N2,787,0.00064,10.61463,13.73659,13.48683,10.73951,0.00049,12.73756,...,0.00149,1.87317,6.61854,4.24585,0.74927,0.000029,24.72585,35.21561,46.70439,49.95122
2,SN002,N2,788,0.00054,10.73951,14.48585,13.98634,11.48878,0.00045,12.86244,...,0.00173,1.62341,5.99415,3.62146,0.74927,0.000028,24.72585,35.21561,46.32976,49.95122
3,SN002,N2,789,0.00059,9.99024,14.73561,14.23610,11.36390,0.00048,13.36195,...,0.00207,0.99902,6.36878,3.24683,0.74927,0.000031,23.85171,35.09073,45.08098,49.95122
4,SN002,N2,791,0.00029,9.24098,16.85854,16.10927,7.86732,0.00029,15.85951,...,0.00119,1.62341,6.24390,3.87122,0.74927,0.000032,25.47512,35.59024,46.57951,49.95122


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51131 entries, 0 to 51130
Data columns (total 78 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Subject                51131 non-null  object 
 1   Sleep_Stage            51131 non-null  object 
 2   Epoch                  51131 non-null  int64  
 3   MeanP_Alpha_F4         51131 non-null  float64
 4   MedianF_Alpha_F4       51131 non-null  float64
 5   MeanF_Alpha_F4         51131 non-null  float64
 6   SpectralEdge_Alpha_F4  51131 non-null  float64
 7   PeakF_Alpha_F4         51131 non-null  float64
 8   MeanP_Beta_F4          51131 non-null  float64
 9   MedianF_Beta_F4        51131 non-null  float64
 10  MeanF_Beta_F4          51131 non-null  float64
 11  SpectralEdge_Beta_F4   51131 non-null  float64
 12  PeakF_Beta_F4          51131 non-null  float64
 13  MeanP_Theta_F4         51131 non-null  float64
 14  MedianF_Theta_F4       51131 non-null  float64
 15  Me

#Converting N1,N2 -> Light_Sleep, N3 -> Deep_Sleep And removing W stage.

In [35]:
df.shape

(51131, 78)

In [36]:
df =df.replace("N1", "Light_Sleep")
df =df.replace("N2", "Light_Sleep")
df =df.replace("N3", "Deep_Sleep")
df =df.replace("R", "REM")
df.drop(df.index[df['Sleep_Stage'] == "W"], inplace = True)


In [37]:
df.shape

(41876, 78)

#Removing columns

In [38]:
df = df.loc[:,df.columns != "Subject"]
df = df.loc[:,df.columns != "Epoch"]


#Encoding target columns (Sleep Stage)

In [39]:
target="Sleep_Stage"

In [40]:
df[target].value_counts()

Light_Sleep    23954
Deep_Sleep      9862
REM             8060
Name: Sleep_Stage, dtype: int64

In [41]:
from sklearn.preprocessing import LabelEncoder
encd = LabelEncoder()

In [42]:
df[target] = encd.fit_transform(df[target])

In [43]:
df[target].value_counts()

1    23954
0     9862
2     8060
Name: Sleep_Stage, dtype: int64

#Feature Selection: ANOVA

In [44]:
num_of_feat = 20

In [45]:

from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import pandas as pd

X = df.loc[:,df.columns != "Sleep_Stage"]
y = df["Sleep_Stage"]

fs = SelectKBest(score_func=f_classif, k=5)
fit = fs.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_ANOVA']

lyst = featureScores.nlargest(num_of_feat,'Score_ANOVA')
ANOVA_feature=list(lyst['Best_columns'])
ANOVA_feature


['MeanF_Alpha_F4',
 'SpectralEdge_Alpha_F4',
 'MeanF_Alpha_C4',
 'MedianF_Alpha_O2',
 'SpectralEdge_Alpha_C4',
 'SpectralEdge_Alpha_O2',
 'MeanF_Alpha_O2',
 'MeanF_Delta_C4',
 'SpectralEdge_Beta_F4',
 'MeanF_Delta_F4',
 'PeakF_Beta_F4',
 'MeanF_Delta_O2',
 'PeakF_Alpha_O2',
 'SpectralEdge_Beta_C4',
 'MeanF_Beta_F4',
 'PeakF_Beta_C4',
 'MedianF_Alpha_F4',
 'MedianF_Beta_F4',
 'SpectralEdge_Theta_O2',
 'SpectralEdge_Gamma_F4']

#Pearson’s

In [46]:

from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import pandas as pd
X = df.loc[:,df.columns != "Sleep_Stage"]
y = df["Sleep_Stage"]
fs = SelectKBest(score_func=f_regression, k=5)
fit = fs.fit(X,y)

#create df for scores
dfscores = pd.DataFrame(fit.scores_)
#create df for column names
dfcolumns = pd.DataFrame(df.columns)
#concat two dataframes for better visualization
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
#naming the dataframe columns
featureScores.columns = ['Best_columns','Score_pearsons']

#print 10 best features
lyst = featureScores.nlargest(num_of_feat,'Score_pearsons')

PEARSON_feature=list(lyst['Best_columns'])
PEARSON_feature


['MedianF_Alpha_F4',
 'MeanF_Alpha_F4',
 'MedianF_Alpha_C4',
 'MeanF_Alpha_C4',
 'MeanP_Alpha_O2',
 'MeanF_Alpha_O2',
 'MedianF_Alpha_O2',
 'SpectralEdge_Beta_F4',
 'SpectralEdge_Alpha_O2',
 'MeanF_Beta_F4',
 'MedianF_Delta_C4',
 'MeanF_Beta_C4',
 'MedianF_Delta_F4',
 'SpectralEdge_Beta_C4',
 'MedianF_Delta_O2',
 'MeanP_Beta_F4',
 'MedianF_Beta_F4',
 'SpectralEdge_Beta_O2',
 'MedianF_Beta_C4',
 'MeanP_Beta_C4']

#Univarient Feature Selection

In [47]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)


#concat two dataframes for better visualization
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns



sorted_df = featureScores.sort_values(by=['Score'], ascending=False)
Univarient_feature = sorted_df["Specs"][:num_of_feat]
Univarient_feature

9             PeakF_Beta_F4
34            PeakF_Beta_C4
8      SpectralEdge_Beta_F4
6           MedianF_Beta_F4
33     SpectralEdge_Beta_C4
7             MeanF_Beta_F4
59            PeakF_Beta_O2
31          MedianF_Beta_C4
3     SpectralEdge_Alpha_F4
2            MeanF_Alpha_F4
56          MedianF_Beta_O2
54           PeakF_Alpha_O2
27           MeanF_Alpha_C4
28    SpectralEdge_Alpha_C4
32            MeanF_Beta_C4
36         MedianF_Theta_C4
42           MeanF_Delta_C4
63    SpectralEdge_Theta_O2
64           PeakF_Theta_O2
58     SpectralEdge_Beta_O2
Name: Specs, dtype: object

#Spliting Dataset into X(features) and y(target)

In [48]:
from scipy.stats import pearson3
#X = df.loc[:,df.columns != target]
#X = df.loc[:,ANOVA_feature]
#X = df.loc[:,Univarient_feature]
X = df.loc[:,PEARSON_feature]
y = df[target]




#Scaling Dataset

In [49]:
X

,MedianF_Alpha_F4,MeanF_Alpha_F4,MedianF_Alpha_C4,MeanF_Alpha_C4,MeanP_Alpha_O2,MeanF_Alpha_O2,MedianF_Alpha_O2,SpectralEdge_Beta_F4,SpectralEdge_Alpha_O2,MeanF_Beta_F4,MedianF_Delta_C4,MeanF_Beta_C4,MedianF_Delta_F4,SpectralEdge_Beta_C4,MedianF_Delta_O2,MeanP_Beta_F4,MedianF_Beta_F4,SpectralEdge_Beta_O2,MedianF_Beta_C4,MeanP_Beta_C4
0,10.11512,17.85756,9.86537,18.48195,0.00023,18.48195,10.61463,27.47317,17.98244,25.35024,0.87415,25.84976,0.74927,27.22341,0.74927,0.00047,17.73268,28.47220,18.85659,0.00054
1,10.61463,13.73659,10.73951,13.86146,0.00044,14.73561,8.99122,20.72976,13.86146,20.23024,1.24878,19.48098,1.12390,20.35512,1.87317,0.00049,12.73756,22.85268,13.11220,0.00063
2,10.73951,14.48585,10.86439,14.73561,0.00034,15.73463,9.36585,21.60390,14.73561,20.97951,0.99902,20.72976,1.12390,21.47902,1.62341,0.00045,12.86244,23.35220,13.73659,0.00053
3,9.99024,14.73561,10.24000,14.86049,0.00042,15.36000,9.11610,22.10341,14.86049,21.60390,1.12390,20.97951,1.24878,21.97854,0.99902,0.00048,13.36195,22.85268,13.36195,0.00064
4,9.24098,16.85854,9.61561,17.60780,0.00029,16.60878,9.24098,24.35122,15.98439,23.35220,1.49854,23.47707,1.37366,24.97561,1.62341,0.00029,15.85951,24.22634,16.73366,0.00039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51126,8.11707,13.48683,7.99220,13.98634,0.00047,14.23610,7.86732,20.48000,13.36195,20.35512,0.74927,20.72976,0.74927,21.10439,0.62439,0.00045,11.98829,21.97854,12.61268,0.00042
51127,7.99220,13.86146,8.11707,14.36098,0.00051,13.98634,7.86732,20.97951,13.23707,20.85463,0.74927,20.72976,0.87415,21.10439,0.74927,0.00042,12.23805,21.97854,12.73756,0.00041
51128,7.99220,13.73659,8.36683,14.11122,0.00050,13.98634,8.11707,20.48000,13.23707,20.10537,0.62439,20.48000,0.74927,20.72976,0.62439,0.00046,12.61268,21.60390,12.86244,0.00043
51129,8.61659,13.23707,8.24195,13.98634,0.00054,13.98634,7.61756,19.48098,13.11220,19.48098,0.87415,20.35512,0.87415,20.60488,0.74927,0.00052,11.73854,21.85366,12.73756,0.00044


In [50]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X)

X[X.columns] =   scaler.transform(X)

In [51]:
X

,MedianF_Alpha_F4,MeanF_Alpha_F4,MedianF_Alpha_C4,MeanF_Alpha_C4,MeanP_Alpha_O2,MeanF_Alpha_O2,MedianF_Alpha_O2,SpectralEdge_Beta_F4,SpectralEdge_Alpha_O2,MeanF_Beta_F4,MedianF_Delta_C4,MeanF_Beta_C4,MedianF_Delta_F4,SpectralEdge_Beta_C4,MedianF_Delta_O2,MeanP_Beta_F4,MedianF_Beta_F4,SpectralEdge_Beta_O2,MedianF_Beta_C4,MeanP_Beta_C4
0,0.553499,0.647889,0.117124,0.428253,-0.079749,0.489376,0.506978,-0.132638,0.502227,-0.152466,-0.230139,-0.199876,-0.231932,-0.209383,-0.224411,-0.067120,-0.037616,-0.206705,-0.087358,-0.071004
1,0.758547,-0.268406,0.288346,-0.294413,-0.054120,-0.137691,0.065785,-0.490081,-0.142219,-0.427053,-0.158427,-0.525910,-0.160424,-0.560077,-0.010144,-0.063479,-0.287198,-0.500355,-0.358403,-0.069735
2,0.809809,-0.101809,0.312807,-0.157692,-0.066324,0.029526,0.167598,-0.443746,-0.005518,-0.386869,-0.206236,-0.461982,-0.160424,-0.502691,-0.057760,-0.070761,-0.280958,-0.474252,-0.328942,-0.071145
3,0.502237,-0.046275,0.190505,-0.138160,-0.056561,-0.033180,0.099724,-0.417269,0.014011,-0.353383,-0.182332,-0.449197,-0.136588,-0.477185,-0.176797,-0.065299,-0.256000,-0.500355,-0.346619,-0.069594
4,0.194668,0.425757,0.068202,0.291532,-0.072426,0.175842,0.133662,-0.298121,0.189769,-0.259621,-0.110618,-0.321340,-0.112751,-0.324155,-0.057760,-0.099889,-0.131209,-0.428574,-0.187527,-0.073118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51126,-0.266693,-0.323940,-0.249784,-0.274881,-0.050459,-0.221299,-0.239656,-0.503320,-0.220333,-0.420355,-0.254044,-0.461982,-0.231932,-0.521819,-0.248219,-0.070761,-0.324636,-0.546033,-0.381972,-0.072695
51127,-0.317952,-0.240641,-0.225325,-0.216286,-0.045578,-0.263104,-0.239656,-0.476843,-0.239862,-0.393566,-0.254044,-0.461982,-0.208096,-0.521819,-0.224411,-0.076222,-0.312156,-0.546033,-0.376080,-0.072836
51128,-0.317952,-0.268406,-0.176403,-0.255349,-0.046798,-0.263104,-0.171782,-0.503320,-0.239862,-0.433749,-0.277948,-0.474768,-0.231932,-0.540948,-0.248219,-0.068940,-0.293438,-0.565610,-0.370188,-0.072554
51129,-0.061642,-0.379474,-0.200864,-0.274881,-0.041916,-0.263104,-0.307533,-0.556274,-0.259390,-0.467236,-0.230139,-0.481161,-0.208096,-0.547324,-0.224411,-0.058017,-0.337114,-0.552559,-0.376080,-0.072413


#Spliting X and y to train and test sets

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20)

In [53]:
y_train.value_counts()

1    19140
0     7907
2     6453
Name: Sleep_Stage, dtype: int64

#Oversampling using SMOTE

**<font color='red'>SNB:</font><font color='olive'> We used *SMOTE* in train data only, so that our test data can remain corrupted data free. The accurecy of the model will be counted only for actual data.</font>**

In [54]:
!pip install imbalanced-learn

In [55]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_train,y_train = smote.fit_resample(X_train,y_train)

In [56]:
y_train.value_counts()

0    19140
1    19140
2    19140
Name: Sleep_Stage, dtype: int64

In [57]:
y_test.value_counts()

1    4814
0    1955
2    1607
Name: Sleep_Stage, dtype: int64

#RandomForest

In [58]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

CPU times: user 30.9 s, sys: 81.1 ms, total: 30.9 s
Wall time: 55.1 s


In [59]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

[[1562  384    9]
 [ 346 4093  375]
 [  14  320 1273]]
              precision    recall  f1-score   support

           0       0.81      0.80      0.81      1955
           1       0.85      0.85      0.85      4814
           2       0.77      0.79      0.78      1607

    accuracy                           0.83      8376
   macro avg       0.81      0.81      0.81      8376
weighted avg       0.83      0.83      0.83      8376

Accurecy:  0.8271251193887297


#AdaBoost

In [60]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier()
ada.fit(X_train, y_train)
y_pred = ada.predict(X_test)

In [61]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

[[1484  418   53]
 [1122 2618 1074]
 [  82  352 1173]]
              precision    recall  f1-score   support

           0       0.55      0.76      0.64      1955
           1       0.77      0.54      0.64      4814
           2       0.51      0.73      0.60      1607

    accuracy                           0.63      8376
   macro avg       0.61      0.68      0.63      8376
weighted avg       0.67      0.63      0.63      8376

Accurecy:  0.6297755491881566


#Gradient_Boosting

In [62]:
from sklearn.ensemble import GradientBoostingClassifier
grd = GradientBoostingClassifier()
grd.fit(X_train, y_train)
y_pred = grd.predict(X_test)

In [63]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

[[1414  518   23]
 [ 559 3672  583]
 [  30  421 1156]]
              precision    recall  f1-score   support

           0       0.71      0.72      0.71      1955
           1       0.80      0.76      0.78      4814
           2       0.66      0.72      0.69      1607

    accuracy                           0.75      8376
   macro avg       0.72      0.74      0.73      8376
weighted avg       0.75      0.75      0.75      8376

Accurecy:  0.7452244508118434


#XGBOOSTING


In [64]:
from xgboost import XGBClassifier

In [65]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [66]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))
print("Accurecy: ",accuracy_score(y_test, y_pred))

[[1513  438    4]
 [ 303 4195  316]
 [  12  410 1185]]
              precision    recall  f1-score   support

           0       0.83      0.77      0.80      1955
           1       0.83      0.87      0.85      4814
           2       0.79      0.74      0.76      1607

    accuracy                           0.82      8376
   macro avg       0.82      0.79      0.80      8376
weighted avg       0.82      0.82      0.82      8376

Accurecy:  0.8229465138490927
